In [2]:
import lmdb
import pickle
import glob
import os
import pandas as pd
from torch_geometric.data import Data

In [3]:
train_dir = "oc22/is2re-total/train"
train_files = sorted(glob.glob(os.path.join(train_dir, "data.*.lmdb")))

# all the properties available in the metadata pickle file
columns = [
    "sid",
    "y_relaxed",
    "bulk_id",
    "bulk_symbols",
    "miller_index",
    "traj_id",
    "slab_sid",
    "ads_symbols",
    "nads"
]

# read metadata file for bulk symbol
with open("oc22_metadata.pkl", "rb") as file:
    metadata = pickle.load(file)

all_data = []

for lmdb_file in train_files:
    env = lmdb.open(lmdb_file, readonly=True, subdir=False, lock=False)

    with env.begin() as txn:
        num_entries = txn.stat()['entries']
        print(f"Processing {lmdb_file}: {num_entries} entries")

        cursor = txn.cursor()

        for key, value in cursor:
            try:
                # load pytorch data obj manually due to the imcompatible PyG version of OC22 dataset
                data = Data.from_dict(pickle.loads(value).__dict__)

                # get target value and sid from training dataset
                sid = data.sid
                y_relaxed = data.y_relaxed

                row = {
                    "sid": sid,
                    "y_relaxed": y_relaxed
                }

                # grab rest of the data from metadata
                if sid in metadata:
                    row.update(metadata[sid])

                all_data.append(row)
            except:
                pass
    env.close()

df = pd.DataFrame(all_data, columns=columns)
df.to_csv("training_FULL.csv", index=False)

Processing oc22/is2re-total/train/data.0000.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0001.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0002.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0003.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0004.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0005.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0006.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0007.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0008.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0009.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0010.lmdb: 1121 entries
Processing oc22/is2re-total/train/data.0011.lmdb: 1120 entries
Processing oc22/is2re-total/train/data.0012.lmdb: 1120 entries
Processing oc22/is2re-total/train/data.0013.lmdb: 1120 entries
Processing oc22/is2re-total/train/data.0014.lmdb: 1120 entries
Processing oc22/is2re-total/train/data.0015.lmdb: 1120 

In [4]:
# get the samples with H as adsorbate
# creates a mask where True if ads is H
contains_h_adsorbate = (df['ads_symbols'] == 'H') & (df['nads'] == 1)

# robust mask for rows with no adsorbates: handles None/NaN, empty lists, and stringified empty lists like '[]'
def _is_no_ads(x):
    # None or NaN
    if pd.isna(x):
        return True
    
    # actual empty list/tuple/set
    if isinstance(x, (list, tuple, set)):
        return len(x) == 0
    
    # string forms: '', '[]', , 'None'
    s = str(x).strip()
    if s == '' or s == '[]' or s.lower() == 'none':
        return True
    
    return False

# creates a mask where False if ads is None
contains_no_ads = df['ads_symbols'].apply(_is_no_ads)

df_only_h = df[contains_h_adsorbate]
df_only_h = df_only_h.reset_index(drop=True)

df_no_ads = df[contains_no_ads]
df_no_ads = df_no_ads.reset_index(drop=True)

df_only_h.to_csv("training_FULL_h.csv", index=False)
df_no_ads.to_csv("training_FULL_no_ads.csv", index=False)

# --------------------------------------------------------------
# randomly sample 100 data points from each dataset
# for the purpose of testing the featurizer. 
# remember to remove this section later

df_only_h_sample = df_only_h.sample(n=100)
df_no_ads_sample = df_no_ads.sample(n=100)
df_only_h_sample.to_csv("training_SAMPLE_h.csv", index=False)
df_no_ads_sample.to_csv("training_SAMPLE_no_ads.csv", index=False)

print(f"size of entire dataset: {len(df)}")
print(f"size of H-ads dataset: {len(df_only_h)}")
print(f"size of no-ads dataset: {len(df_no_ads)}")
print(f"sample size: n = {len(df_only_h_sample)}")

size of entire dataset: 45890
size of H-ads dataset: 1605
size of no-ads dataset: 14646
sample size: n = 100


In [5]:
# featurization
# please specify whether to featurize sample or full dataset:
h = df_only_h # use df_only_h to process full dataset
no_ads = df_no_ads # use df_no_ads to process full dataset

h_clean = h[['bulk_id', 'bulk_symbols', 'y_relaxed']]
no_ads_clean = no_ads[['bulk_id', 'bulk_symbols', 'y_relaxed']]

print(h_clean.head())
print(no_ads_clean.head())

         bulk_id bulk_symbols   y_relaxed
0  SbPdO4-rutile     Sb1Pd1O4 -531.043770
1        mp-8086     Na4Zn4O6 -135.378886
2     mp-1183298     Ba1Re1O3 -538.652121
3  FePdO4-rutile     Fe1Pd1O4 -387.021855
4        mp-2133        Zn2O2 -193.572049
      bulk_id bulk_symbols   y_relaxed
0    mp-11019     Ga2Cu2O4 -603.007638
1   mp-756297   Zn8In16O32 -576.660659
2     mp-2697        Sr2O4 -256.748952
3  mp-1183971     Cs1Ru1O3 -319.483107
4   mp-867628    Y6Cu11O24 -451.671846


In [9]:
# convert bulk symbols to Composition object
from pymatgen.core import Composition

pd.options.mode.copy_on_write = True
h_clean['bulk_symbols'] = h_clean['bulk_symbols'].apply(Composition)
no_ads_clean['bulk_symbols'] = no_ads_clean['bulk_symbols'].apply(Composition)

print("--- H-ads ---")
for c in h_clean['bulk_symbols'].head():
    print(f"formula: {c.formula}; num of atoms: {c.num_atoms}")

print("\n--- no-ads ---")
for c in no_ads_clean['bulk_symbols'].head():
    print(f"formula: {c.formula}; num of atoms: {c.num_atoms}")

--- H-ads ---
formula: Sb1 Pd1 O4; num of atoms: 6.0
formula: Na4 Zn4 O6; num of atoms: 14.0
formula: Ba1 Re1 O3; num of atoms: 5.0
formula: Fe1 Pd1 O4; num of atoms: 6.0
formula: Zn2 O2; num of atoms: 4.0

--- no-ads ---
formula: Ga2 Cu2 O4; num of atoms: 8.0
formula: Zn8 In16 O32; num of atoms: 56.0
formula: Sr2 O4; num of atoms: 6.0
formula: Cs1 Ru1 O3; num of atoms: 5.0
formula: Y6 Cu11 O24; num of atoms: 41.0


In [ ]:
from matminer.featurizers.composition import WenAlloys
from matminer.featurizers.composition import ElementProperty
from matminer.featurizers.composition import ElementFraction
from matminer.featurizers.composition import TMetalFraction
from matminer.featurizers.composition import Stoichiometry
from matminer.featurizers.composition import BandCenter

def featurize_df(df):
    featurized_df = df.copy()
    print("Initial df shape:", featurized_df.shape)
    
    features = [
        'AtomicWeight',
        'MeltingT',
        'CovalentRadius',
        'Electronegativity',
        'NsValence',
        'NpValence',
        'NdValence',
        'NfValence',
        'NValence',
        'NsUnfilled',
        'NpUnfilled',
        'NdUnfilled',
        'NfUnfilled',
        'NUnfilled',
        'GSvolume_pa',
        'GSbandgap',
        'GSmagmom'
    ]
    
    # this code looks janky because I was trying to fix a weird behaviour left over
    # from when I tried to use multiprocessing to speed up the featurization.
    
    f = ElementProperty('magpie', features=features, stats=['mean'], impute_nan=False)
    f.set_n_jobs(1)

    featurized_df = f.featurize_dataframe(featurized_df, col_id="bulk_symbols", ignore_errors=True, return_errors=True)
    print("After ElementProperty:", type(featurized_df), featurized_df.shape if featurized_df is not None else "None")
    
    f = WenAlloys(impute_nan=False)
    f.set_n_jobs(1)

    featurized_df = f.featurize_dataframe(featurized_df, col_id="bulk_symbols", ignore_errors=True)
    print("After WenAlloys:", type(featurized_df), featurized_df.shape if featurized_df is not None else "None")
    
    f = ElementFraction()
    f.set_n_jobs(1)

    featurized_df = f.featurize_dataframe(featurized_df, col_id="bulk_symbols", ignore_errors=True)
    print("After ElementFraction:", type(featurized_df), featurized_df.shape if featurized_df is not None else "None")
    
    f = TMetalFraction()
    f.set_n_jobs(1)

    featurized_df = f.featurize_dataframe(featurized_df, col_id="bulk_symbols", ignore_errors=True)
    print("After TMetalFraction:", type(featurized_df), featurized_df.shape if featurized_df is not None else "None")
    
    f = Stoichiometry()
    f.set_n_jobs(1)

    featurized_df = f.featurize_dataframe(featurized_df, col_id="bulk_symbols", ignore_errors=True)
    print("After Stoichiometry:", type(featurized_df), featurized_df.shape if featurized_df is not None else "None")
    
    f = BandCenter(impute_nan=False)
    f.set_n_jobs(1)

    featurized_df = f.featurize_dataframe(featurized_df, col_id="bulk_symbols", ignore_errors=True)
    print("After BandCenter:", type(featurized_df), featurized_df.shape if featurized_df is not None else "None")
    
    return featurized_df

In [11]:
import warnings


# suppress annoying warnings
warnings.filterwarnings('ignore', message='.*impute_nan.*')

# warning: this step can take a while!
h_featurized = featurize_df(h_clean)
no_ads_featurized = featurize_df(no_ads_clean)

Initial df shape: (1605, 3)


ElementProperty:   0%|          | 0/1605 [00:00<?, ?it/s]

After ElementProperty: <class 'pandas.core.frame.DataFrame'> (1605, 21)


WenAlloys:   0%|          | 0/1605 [00:00<?, ?it/s]

After WenAlloys: <class 'pandas.core.frame.DataFrame'> (1605, 46)


...

After TMetalFraction: <class 'pandas.core.frame.DataFrame'> (14646, 165)


Stoichiometry:   0%|          | 0/14646 [00:00<?, ?it/s]

After Stoichiometry: <class 'pandas.core.frame.DataFrame'> (14646, 171)


BandCenter:   0%|          | 0/14646 [00:00<?, ?it/s]

After BandCenter: <class 'pandas.core.frame.DataFrame'> (14646, 172)


In [14]:
h_featurized.head()

,bulk_id,bulk_symbols,y_relaxed,MagpieData mean AtomicWeight,MagpieData mean MeltingT,MagpieData mean CovalentRadius,MagpieData mean Electronegativity,MagpieData mean NsValence,MagpieData mean NpValence,MagpieData mean NdValence,...,Ts,Og,transition metal fraction,0-norm,2-norm,3-norm,5-norm,7-norm,10-norm,band center
0,SbPdO4-rutile,"(Sb, Pd, O)",-531.043770,48.696267,491.838333,90.333333,3.001667,1.666667,3.166667,3.333333,...,0,0,0.166667,3,0.707107,0.673540,0.666927,0.666678,0.666667,6.416770
1,mp-8086,"(Na, Zn, O)",-135.378886,32.105391,327.357143,110.571429,2.211429,1.714286,1.714286,2.857143,...,0,0,0.000000,3,0.589015,0.500485,0.449086,0.435403,0.430035,4.984547
2,mp-1183298,"(Ba, Re, O)",-538.652121,74.306440,924.680000,112.800000,2.622000,2.000000,2.400000,1.000000,...,0,0,0.200000,3,0.663325,0.614463,0.600984,0.600078,0.600002,5.403931
3,FePdO4-rutile,"(Fe, Pd, O)",-387.021855,37.710433,643.041667,89.166667,2.965000,1.666667,2.666667,2.666667,...,0,0,0.333333,3,0.707107,0.673540,0.666927,0.666678,0.666667,6.220992
4,mp-2133,"(Zn, O)",-193.572049,40.689700,373.740000,94.000000,2.545000,2.000000,2.000000,5.000000,...,0,0,0.000000,2,0.707107,0.629961,0.574349,0.552045,0.535887,5.951264


In [15]:
no_ads_featurized.head()

,bulk_id,bulk_symbols,y_relaxed,MagpieData mean AtomicWeight,MagpieData mean MeltingT,MagpieData mean CovalentRadius,MagpieData mean Electronegativity,MagpieData mean NsValence,MagpieData mean NpValence,MagpieData mean NdValence,...,Ts,Og,transition metal fraction,0-norm,2-norm,3-norm,5-norm,7-norm,10-norm,band center
0,mp-11019,"(Ga, Cu, O)",-603.007638,41.316950,442.570000,96.500000,2.647500,1.750000,2.250000,5.000000,...,0,0,0.250000,3,0.612372,0.538609,0.506099,0.501109,0.500098,5.320929
1,mp-756297,"(Zn, In, O)",-576.660659,51.287657,253.054286,95.714286,2.710000,2.000000,2.571429,4.285714,...,0,0,0.000000,3,0.654654,0.597048,0.575065,0.572069,0.571484,5.437784
2,mp-2697,"(Sr, O)",-256.748952,39.872933,386.533333,109.000000,2.610000,2.000000,2.666667,0.000000,...,0,0,0.000000,2,0.745356,0.693361,0.670782,0.667408,0.666732,5.466758
3,mp-1183971,"(Cs, Ru, O)",-319.483107,56.394730,614.598000,117.600000,2.662000,1.600000,2.400000,1.400000,...,0,0,0.200000,3,0.663325,0.614463,0.600984,0.600078,0.600002,5.235859
4,mp-867628,"(Y, Cu, O)",-451.671846,39.425041,659.626098,101.853659,2.701951,1.731707,2.341463,2.829268,...,0,0,0.268293,3,0.660341,0.606434,0.587827,0.585726,0.585390,5.799601


In [16]:
h_featurized.to_csv("training_FULL_h_comp_featurized_.csv")
no_ads_featurized.to_csv("training_FULL_no_ads_comp_featurized_.csv")

possible MP fields that can be requested by API:

['builder_meta',
 'nsites',
 'elements',
 'nelements',
 'composition',
 'composition_reduced',
 'formula_pretty',
 'formula_anonymous',
 'chemsys',
 'volume',
 'density',
 'density_atomic',
 'symmetry',
 'deprecated',
 'deprecation_reasons',
 'last_updated',
 'origins',
 'warnings',
 'property_name',
 'task_ids',
 'uncorrected_energy_per_atom',
 'formation_energy_per_atom',
 'energy_above_hull',
 'is_stable',
 'equilibrium_reaction_energy_per_atom',
 'decomposes_to',
 'xas',
 'grain_boundaries',
 'band_gap',
 'cbm',
 'vbm',
 'efermi',
 'is_gap_direct',
 'is_metal',
 'es_source_calc_id',
 'bandstructure',
 'dos',
 'dos_energy_up',
 'dos_energy_down',
 'is_magnetic',
 'ordering',
 'total_magnetization',
 'total_magnetization_normalized_vol',
 'total_magnetization_normalized_formula_units',
 'num_magnetic_sites',
 'num_unique_magnetic_sites',
 'types_of_magnetic_species',
 'bulk_modulus',
 'shear_modulus',
 'universal_anisotropy',
 'homogeneous_poisson',
 'e_total',
 'e_ionic',
 'e_electronic',
 'n',
 'e_ij_max',
 'weighted_surface_energy_EV_PER_ANG2',
 'weighted_surface_energy',
 'weighted_work_function',
 'surface_anisotropy',
 'shape_factor',
 'has_reconstructed',
 'possible_species',
 'has_props',
 'theoretical',
 'database_IDs']

DensityFeatures:
- density
- vpa
- packing fraction

GlobalSymmetryFeatures:
- spacegroup_num
- crystal_system
- crystal_system_int
- is_centrosymmetric
- n_symmetry_ops

In [ ]:
# get structure properties
from mp_api.client import MPRester
from pymatgen.core import Structure
from matminer.featurizers.structure import DensityFeatures, GlobalSymmetryFeatures, EwaldEnergy


key = "H1TcFfkjPB8IR4vjQB1vXdZ7CbceH5I4"

density_featurizer = DensityFeatures()
global_symmetry_featurizer = GlobalSymmetryFeatures()
ewald_energy_featurizer = EwaldEnergy()

def featurize_structure(df):
    structure_features = []
    structure_features_cols = [
        'bulk_symbols',
        'density', 
        'vpa', 
        'packing_fraction', 
        'spacegroup_num', 
        'crystal_system', 
        'crystal_system_int', 
        'is_centrosymmetric', 
        'n_symmetry_ops']
    
    with MPRester(key) as mpr:
        for i, row in df.iterrows():
            bulk_symbol = row['bulk_symbols'] # already a Composition object
            formula = bulk_symbol.reduced_formula

            try:
                docs = mpr.materials.summary.search(formula=formula, fields=['material_id', 'structure', 'energy_per_atom'])
                if docs:
                    # get the structure with the lowest energy/atom
                    best_doc = min(docs, key=lambda x: x.energy_per_atom)
                    best_structure = best_doc.structure

                    # add featurizers here if needed
                    density_features = density_featurizer.featurize(best_structure)
                    global_symmetry_features = global_symmetry_featurizer.featurize(best_structure)

                    structure_features.append([bulk_symbol] + density_features + global_symmetry_features)
                else:
                    # append empty vals if structure cannot be found
                    structure_features.append([bulk_symbol] + ([None] * 8))
            except:
                pass

    return pd.DataFrame(data=structure_features, columns=structure_features_cols)


In [ ]:
h_structure_featurized = featurize_structure(h_clean)
no_ads_structure_featurized = featurize_structure(no_ads_clean)

Retrieving SummaryDoc documents: 0it [00:00, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving SummaryDoc documents: 0it [00:00, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/12 [00:00<?, ?it/s]

...

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/9 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
h_structure_featurized.head()

,bulk_symbols,density,vpa,packing_fraction,spacegroup_num,crystal_system,crystal_system_int,is_centrosymmetric,n_symmetry_ops
0,"(Li, Sb, O)",3.544145,8.913568,0.862161,1.0,triclinic,7.0,False,2.0
1,"(Ta, Bi, O)",9.283679,14.454630,0.423727,2.0,triclinic,7.0,True,4.0
2,"(Cs, Mn, O)",3.649580,19.097724,0.774397,62.0,orthorhombic,5.0,True,16.0
3,"(Sr, Bi, O)",7.355103,15.559794,0.686152,14.0,monoclinic,6.0,True,8.0
4,"(K, Au, O)",3.845922,28.519092,1.016985,221.0,cubic,1.0,True,96.0
...,...,...,...,...,...,...,...,...,...
95,"(Mn, Sn, O)",NaN,NaN,NaN,NaN,None,NaN,None,NaN
96,"(Cr, Au, O)",NaN,NaN,NaN,NaN,None,NaN,None,NaN
97,"(Y, Au, O)",7.993944,16.507421,0.553457,194.0,hexagonal,2.0,True,48.0
98,"(Ca, Os, O)",7.839020,11.790716,0.616521,62.0,orthorhombic,5.0,True,8.0


In [ ]:
no_ads_structure_featurized.head()

In [ ]:
# combine composition and structure features
h_final = pd.merge(h_featurized, h_structure_featurized, on='bulk_symbols')
no_ads_final = pd.merge(no_ads_featurized, no_ads_structure_featurized, on='bulk_symbols')

# here save the final fully featurized dataset.
# rename the file if this is the full dataset.
# remember to check the dataset one more time when implementing the ML pipeline.
h_final.to_csv("training_FULL_h_featurized.csv", index=False)
no_ads_final.to_csv("training_FULL_no_ads_featurized.csv", index=False)

In [ ]:
h_final.head()

,bulk_id,bulk_symbols,y_relaxed,MagpieData mean AtomicWeight,MagpieData mean MeltingT,MagpieData mean CovalentRadius,MagpieData mean Electronegativity,MagpieData mean NsValence,MagpieData mean NpValence,MagpieData mean NdValence,...,10-norm,band center,density,vpa,packing_fraction,spacegroup_num,crystal_system,crystal_system_int,is_centrosymmetric,n_symmetry_ops
0,mp-37436,"(Li, Sb, O)",-478.520592,19.024529,314.886429,102.214286,2.110714,1.500000,1.928571,0.714286,...,0.509793,4.612597,3.544145,8.913568,0.862161,1.0,triclinic,7.0,False,2.0
1,mp-29680,"(Ta, Bi, O)",-777.683692,80.812398,550.611765,97.529412,2.877647,2.000000,3.294118,2.705882,...,0.647061,6.087441,9.283679,14.454630,0.423727,2.0,triclinic,7.0,True,4.0
2,mp-18994,"(Cs, Mn, O)",-254.609219,41.973516,339.965000,107.833333,2.683333,1.833333,2.666667,0.833333,...,0.666667,5.450709,3.649580,19.097724,0.774397,62.0,orthorhombic,5.0,True,16.0
3,mp-29164,"(Sr, Bi, O)",-681.089878,68.919720,351.760000,108.200000,2.658000,2.000000,3.000000,2.000000,...,0.600002,5.508122,7.355103,15.559794,0.686152,14.0,monoclinic,6.0,True,8.0
4,mp-9200,"(K, Au, O)",-148.816550,66.052174,480.344000,162.200000,1.688000,1.200000,0.800000,2.000000,...,0.600002,3.614867,3.845922,28.519092,1.016985,221.0,cubic,1.0,True,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,MnSnO4-rutile,"(Mn, Sn, O)",-484.053953,39.607608,373.880000,90.333333,2.878333,2.000000,3.000000,2.500000,...,0.666667,6.085708,NaN,NaN,NaN,NaN,None,NaN,None,NaN
104,CrAuO4-rutile,"(Cr, Au, O)",-519.243940,52.160045,622.755000,89.833333,2.993333,1.666667,2.666667,2.500000,...,0.666667,6.408714,NaN,NaN,NaN,NaN,None,NaN,None,NaN
105,mp-30251,"(Y, Au, O)",-562.497070,79.467805,811.482500,114.500000,2.660000,1.750000,2.000000,2.750000,...,0.500098,5.719124,7.993944,16.507421,0.553457,194.0,hexagonal,2.0,True,48.0
106,mp-1016900,"(Ca, Os, O)",-193.658560,55.661240,917.080000,103.600000,2.704000,2.000000,2.400000,1.200000,...,0.600002,5.779753,7.839020,11.790716,0.616521,62.0,orthorhombic,5.0,True,8.0


In [ ]:
no_ads_final.head()